In [8]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from climada.util.constants import SYSTEM_DIR

# --- PATHS ---
haz_dir = SYSTEM_DIR / "hazard" / "future" / "CHAZ_update" / "maps"
shapefile_path = SYSTEM_DIR / "ne_10m_land" / "ne_10m_land.shp"
netcdf_path = haz_dir / f"TC_100.0_180.0_5.0_60.0_0300as_CHAZ_ERA5_exceedance_intensity.nc"

var_key = "rp_100"

# --- LOAD LAND SHAPE ---
land = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

# --- LOAD DATA ---
ds = xr.open_dataset(netcdf_path)
lat = ds["lat"].values
lon = ds["lon"].values
values = ds[var_key].values

In [9]:
ds

<xarray.Dataset> Size: 9MB
Dimensions:  (points: 142731)
Coordinates:
    lon      (points) float64 1MB 172.5 172.6 172.7 172.8 ... 177.5 178.5 179.5
    lat      (points) float64 1MB 52.96 52.96 52.96 52.96 ... 59.5 59.5 59.5
Dimensions without coordinates: points
Data variables:
    rp_10    (points) float64 1MB ...
    rp_25    (points) float64 1MB ...
    rp_50    (points) float64 1MB ...
    rp_100   (points) float64 1MB 38.76 39.21 39.11 39.14 ... 24.94 23.96 23.75
    rp_250   (points) float64 1MB ...
    rp_1000  (points) float64 1MB ...

In [12]:
import xarray as xr
import pandas as pd
from climada.util.constants import SYSTEM_DIR

# — Paths —
hazard_dir = SYSTEM_DIR / "hazard" / "future" / "CHAZ_update" / "maps"
#int_path   = hazard_dir / "TC_100.0_180.0_5.0_60.0_0300as_CHAZ_ERA5_exceedance_intensity.nc"
int_path   = hazard_dir /"TC_global_0300as_CHAZ_ALL-MODELS_fut2_ssp245_480ens_SD_H08_exceedance_intensity.nc"

# — Load dataset —
ds_int = xr.open_dataset(int_path)

# --- Manila coords and nearest‐point index ---
lon_t, lat_t = 121.0, 14.6
lons = ds["lon"].values
lats = ds["lat"].values
# Euclidean distance in lon/lat space
idx = np.argmin((lons - lon_t)**2 + (lats - lat_t)**2)

# --- Build the table of all rp_* variables ---
# collect and sort your rp variable names
rp_vars = [v for v in ds.data_vars if v.startswith("rp_")]
rp_vars = sorted(rp_vars, key=lambda v: int(v.split("_")[1]))

rows = []
for var in rp_vars:
    rp = int(var.split("_")[1])
    intensity = float(ds[var].values[idx])
    rows.append({
        "Return Period (yr)": rp,
        "Exceedance Intensity (m/s)": round(intensity, 2)
    })

df_manila = pd.DataFrame(rows)
display(df_manila)

,Return Period (yr),Exceedance Intensity (m/s)
0,10,36.12
1,25,43.02
2,50,47.49
3,100,51.59
4,250,57.60
5,1000,62.87


In [13]:
# --- Define your cities of interest ---
cities = {
    "Noumea":    {"lon": 166.45,  "lat": -22.28},  # Southwest Pacific
    "Miami":     {"lon": -80.2,   "lat": 25.8},    # North Atlantic
    "Acapulco":  {"lon": -99.9,   "lat": 16.8},    # East Pacific
    "Toamasina": {"lon": 49.37,   "lat": -18.15},  # Southwest Indian (Madagascar)
    "Mumbai":    {"lon": 72.8,    "lat": 18.9},    # North Indian
    "Manila":   {"lon": 121.0,   "lat": 14.6}    # Northwest Pacific
}

# --- Find nearest‐point index for each city once ---
coords = [(c["lon"], c["lat"]) for c in cities.values()]
lons = ds["lon"].values
lats = ds["lat"].values

def nearest_idx(lon_t, lat_t):
    dist2 = (lons - lon_t)**2 + (lats - lat_t)**2
    return np.argmin(dist2)

city_idxs = {name: nearest_idx(c["lon"], c["lat"]) for name,c in cities.items()}

# --- Build a long-form table of RP vs. intensity ---
rp_vars = sorted([v for v in ds.data_vars if v.startswith("rp_")],
                 key=lambda v: int(v.split("_")[1]))

rows = []
for city, idx in city_idxs.items():
    for var in rp_vars:
        rp = int(var.split("_")[1])
        intensity = float(ds[var].values[idx])
        rows.append({
            "City": city,
            "Return Period (yr)": rp,
            "Exceedance Intensity (m/s)": round(intensity, 2)
        })

df = pd.DataFrame(rows)

# --- Pivot to wide form ---
table = df.pivot(index="City", 
                 columns="Return Period (yr)", 
                 values="Exceedance Intensity (m/s)")
display(table)


Return Period (yr),10,25,50,100,250,1000
City,,,,,,
Acapulco,0.00,23.44,27.37,30.57,34.55,39.67
Manila,36.12,43.02,47.49,51.59,57.60,62.87
Miami,0.00,0.00,0.00,0.00,0.00,0.00
Mumbai,0.00,0.00,0.00,19.87,24.51,32.05
Noumea,0.00,22.33,28.28,36.03,47.83,55.17
Toamasina,26.38,36.44,42.46,46.42,52.53,58.08


In [15]:
# --- Buffer: k nearest neighbors ---
k = 5  # number of nearest points to average
lons = ds["lon"].values
lats = ds["lat"].values

# Precompute k neighbors for each city
city_neighbors = {}
for name, crd in cities.items():
    lon_t, lat_t = crd['lon'], crd['lat']
    d2 = (lons - lon_t)**2 + (lats - lat_t)**2
    idxs = np.argsort(d2)[:k]
    city_neighbors[name] = idxs

# --- Gather results ---
rp_vars = sorted([v for v in ds.data_vars if v.startswith('rp_')],
                 key=lambda v: int(v.split('_')[1]))
rows = []
for city, idxs in city_neighbors.items():
    for var in rp_vars:
        rp = int(var.split('_')[1])
        vals = ds[var].values[idxs]
        mean_int = np.nanmean(vals)
        rows.append({
            'City': city,
            'Return Period (yr)': rp,
            'Avg Exceedance Intensity (m/s)': round(float(mean_int), 2)
        })

# --- Format table ---
df = pd.DataFrame(rows)
table = df.pivot(index='City',
                 columns='Return Period (yr)',
                 values='Avg Exceedance Intensity (m/s)')
display(table)

Return Period (yr),10,25,50,100,250,1000
City,,,,,,
Acapulco,7.11,23.43,27.37,30.74,34.59,40.00
Manila,36.10,42.92,47.48,51.77,57.37,63.26
Miami,0.00,0.00,0.00,0.00,0.00,0.00
Mumbai,0.00,0.00,0.00,19.93,24.55,32.13
Noumea,0.00,22.78,29.88,36.46,45.72,56.81
Toamasina,26.26,36.31,41.95,46.91,52.61,59.82


In [18]:
# --- Cell 2: Return Periods for thr_33 & thr_50 (future ssp245 dataset) ---
# Load return-period dataset for SSP2-4.5 scenario
rp_path = hazard_dir / "TC_global_0300as_CHAZ_ALL-MODELS_fut2_ssp245_480ens_SD_H08_return_periods.nc"
ds_rp = xr.open_dataset(rp_path)

# Extract rp_33 and rp_50 for each city (nearest point)
cols = ['City', 'thr_33 (yr)', 'thr_50 (yr)']
rp_results = []
for city, crd in cities.items():
    # find nearest index
    d2 = (ds_rp['lon'].values - crd['lon'])**2 + (ds_rp['lat'].values - crd['lat'])**2
    idx = np.argmin(d2)
    rp33 = float(ds_rp['thr_33'].values[idx])
    rp50 = float(ds_rp['thr_50'].values[idx])
    rp_results.append({
        'City': city,
        'thr_33 (yr)': round(rp33,1),
        'thr_50 (yr)': round(rp50,1)
    })
# Create DataFrame and display
rp_df = pd.DataFrame(rp_results)[cols].set_index('City')
display(rp_df)

,thr_33 (yr),thr_50 (yr)
City,,
Noumea,11.3,48.8
Miami,17.1,173.2
Acapulco,22.0,536.8
Toamasina,9.4,50.2
Mumbai,132.9,539.9
Manila,9.2,101.0
